In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

import glob

In [2]:
def determine_number_of_models(folder_to_search):
    folders = [f for f in glob.glob(folder_to_search + "**/", recursive = True)]
    model_location = []
    model_names = []
    for folderscounter in folders:
        model_location = [f for f in glob.glob(folderscounter + "/*tst_pred*", recursive = True)]
    if len(model_location) > 0:
        for cnt in model_location:
            split_str = cnt.split('\\')
            model_names.append(split_str[-1][:-4])
    
    return len(model_location), model_location, model_names

In [3]:
def load_excel(path):
    dfoutput = pd.read_excel(path, parse_dates = ['Failure Time'])
    return dfoutput

In [4]:
def load_csv(path):
    dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    dfoutput = pd.read_csv(path, parse_dates = ['timestamp'], date_parser = dateparse)
    dfoutput.set_index(['timestamp'], inplace = True)
#    dfoutput['date'] = dfoutput.index
    return dfoutput

In [5]:
def no_of_alerts_before_failure(df_failure, alert_results, model_name, output):
    for cnt in range(len(df_failure)):                   
        right_limit = df_failure.loc[cnt,'Failure Time']
        left_limit = df_failure.loc[cnt,'Failure Time'] - timedelta(hours = 14*24)
        sampdf = alert_results[(alert_results.index >= left_limit) & (alert_results.index < right_limit)]
        sampdf = sampdf[(sampdf['system_status'] == 'online') & (sampdf['prediction'] == -1) & (sampdf['alert'] == 1)]
        output.loc[cnt, model_name] = len(sampdf) 
    return output

In [6]:
def timestamp_of_each_alert(df_failure, alert_results, model_name, output):
    out = ''
    for cnt in range(0, len(df_failure)):                   
        right_limit = df_failure.loc[cnt,'Failure Time']
        left_limit = df_failure.loc[cnt,'Failure Time'] - timedelta(hours = 14*24)
        sampdf = alert_results[(alert_results.index >= left_limit) & (alert_results.index < right_limit)]
        sampdf = sampdf[(sampdf['system_status'] == 'online') & (sampdf['prediction'] == -1) & (sampdf['alert'] == 1)]
        for j in range(len(sampdf)):
            output.loc[cnt, model_name + '_TS'] = sampdf.index[j]
            out = out + (output.loc[cnt, model_name + '_TS']) + ';'
                    
    return out

In [7]:
def isintop10(df_rca, sampdf, listofrcatags):
    out = ''
    if isinstance(listofrcatags, str):
        splittags = listofrcatags.split(';')       
        for i in range(len(sampdf)): 
#            print(i, len(sampdf))
            templist = []
            res = 0
            for j in range(10):
                templist.append(df_rca.loc[sampdf.index[i], 'top' + str(j+1)])           
            for tagname in templist:
                if tagname in splittags:
                    res = 1     
            if res == 1:
                out = out + 'y' + ';'
            if res == 0:
                out = out + 'n' + ';'           
    return out
    
    
def evaluate_presence_of_root_cause_tags(df_failure, alert_results, model_name, df_rca, output):  
    for cnt in range(0, len(df_failure)):
        out = ''
        right_limit = df_failure.loc[cnt,'Failure Time']
        left_limit = df_failure.loc[cnt,'Failure Time'] - timedelta(hours = 14*24)
        sampdf = alert_results[(alert_results.index >= left_limit) & (alert_results.index < right_limit)]
        sampdf = sampdf[(sampdf['system_status'] == 'online') & (sampdf['prediction'] == -1) & (sampdf['alert'] == 1)]
        output.loc[cnt, model_name + '_relevanttags'] = isintop10(df_rca, sampdf, df_failure.loc[cnt,'RCA_Tags'])
        timeind =str()
        for count in range(len(sampdf)):
            timind = timeind + str(sampdf.index[count]) +';'
            date_time = sampdf.index[count].strftime("%m/%d/%Y %H:%M:%S")
            timeind = timeind + date_time + ';'
        output.loc[cnt, model_name + '_TS'] = timeind
    return output


In [8]:
def create_model_ids(model):
    mid = []
    for i in range(0, no_of_models):
        m = 'M' + str(i)
        mid.append(m)
    return mid      

def create_list(a, b, n):
    output = ''
    for i in range(int(n)):
        output = output + str(a) + '.' + str(b) + '.' + str(i+1) + ';'
    return output

def generate_alert_ids(m, model_name, output):
    for j in range(0, len(output)):
        xx = create_list(m, j+1, output.loc[j, model_name])
        output.loc[j, model_name + '_AlertIds'] = xx
                               
    return output


In [9]:
def split_alerts_tags(df, model_name):
    output = pd.DataFrame()
    a = df[model_name + '_AlertIds'].str.split(';')
    b = df[model_name + '_TS'].str.split(';')
    c = df[model_name + '_relevanttags'].str.split(';')
    k=0
    for i in range(len(a)):
        if a[i] != ['']:
            for j in range(len(a[i])):
                if len(a[i][j]) > 0 :
                    output.loc[k,'AlertIds'] = a[i][j]
                    output.loc[k,'TimeStamp'] = b[i][j]
                    
                    output.loc[k,'Failure Time'] = df.loc[i,'Failure Time']
                    output.loc[k,'Actual_Tag'] = df.loc[i,'RCA_Tags']
                    if c[i] != ['']:
                        output.loc[k,'Relevant_Tag'] = c[i][j]
                    
                    
                k=k+1
            
    output['TimeStamp'] = pd.to_datetime(output['TimeStamp'], format='%m/%d/%Y %H:%M:%S')
    return output
    
def populate_top_10(df,df_rca):
    for cnt in range(len(df)):
        ts = df.loc[df.index[cnt],'TimeStamp']
        top10=str()
        top10 = df_rca.loc[ts,'top1']
        top10 = top10 + ';' + df_rca.loc[ts,'top2']
        top10 = top10 + ';' + df_rca.loc[ts,'top3']
        top10 = top10 + ';' + df_rca.loc[ts,'top4']
        top10 = top10 + ';' + df_rca.loc[ts,'top5']
        top10 = top10 + ';' + df_rca.loc[ts,'top6']
        top10 = top10 + ';' + df_rca.loc[ts,'top7']
        top10 = top10 + ';' + df_rca.loc[ts,'top8']
        top10 = top10 + ';' + df_rca.loc[ts,'top9']
        top10 = top10 + ';' + df_rca.loc[ts,'top10']
        df.loc[df.index[cnt],'Top10_Tag'] = top10
    return df
 
def find_rank(actual, model):
    actual = str(actual).split(';')
    model = str(model).split(';')
    pos = ''
    value = ''
    for cnt in actual:
        if cnt in model:
            pos = pos + str(model.index(cnt)) + ';'
            value = value + str(cnt) + ';'
    return pos, value

def populate_ranking(df):
    for cnt in range(len(df)):
        df.loc[df.index[cnt],'Matched_Tag'],df.loc[df.index[cnt],'Ranking'] = find_rank(df.loc[df.index[cnt],'Actual_Tag'],df.loc[df.index[cnt],'Top10_Tag'])
    return df
    

# MAIN PROGRAM STARTS HERE

In [10]:
# Specify the folder where the results file are placed
folder_to_search = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\ONEGAS_LPC4\KPI'
# folder_to_search = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\GT5 Results'

no_of_models, model_location, model_names = determine_number_of_models(folder_to_search)
# Specify the input failure information file validated with the SMEs
location_of_failure_file = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\ONEGAS_LPC4\KPI\NL_ONEGAS_LPC4_trips_with add info.xlsx'
# location_of_failure_file = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\GT5 Results\GT5-downtime.xlsx'
df_failure = load_excel(location_of_failure_file)

# Specify the input rca file 
location_of_rca_file = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\ONEGAS_LPC4\KPI\NL_ONEGAS_LPC4_w1d_01May2019_20Mar2020_rca_tag_rank.csv'
# location_of_rca_file = r'C:\Users\S.Chandrasekaran5\Shell\Kumar, Jitendra SSSCCH-FO XF - PAM Research Findings\Qualitative KPI\GT5 Results\Brunei_Champion_GT5_w1d_rca_by_rolling_z_score_01Aug2018_01Jul2019_rca_tag_rank.csv'
df_rca = load_csv(location_of_rca_file)

In [11]:
# Determine the first result:  Alert_Analysis_A
k=0
analysis_result_df = df_failure
mid = create_model_ids(no_of_models)
for cnt in model_location:
    print('Processing Model : ', model_names[k])
    alert_results = load_csv(cnt)
    analysis_result_df = no_of_alerts_before_failure(df_failure, alert_results, model_names[k], analysis_result_df)
    analysis_result_df = generate_alert_ids(mid[k], model_names[k], analysis_result_df)
    analysis_result_df = evaluate_presence_of_root_cause_tags(df_failure, alert_results, model_names[k], df_rca, analysis_result_df)
    k = k+1

Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.05_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.05_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m4hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020


In [12]:
# Determine the Second result: Alert_Analysis_B
deep_analysis_result_df = pd.DataFrame()
for cnt in range(len(model_names)):
    print('Processing Model : ', model_names[cnt])
    res = split_alerts_tags(analysis_result_df, model_names[cnt])
    if len(deep_analysis_result_df) == 0:
        deep_analysis_result_df = res
    else:
        deep_analysis_result_df = deep_analysis_result_df.append(res, ignore_index = True)


deep_analysis_result_df = populate_top_10(deep_analysis_result_df,df_rca)
deep_analysis_result_df = populate_ranking(deep_analysis_result_df)

Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.05_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.001_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.05_01May2019_20Mar2020
Processing Model :  NL_ONEGAS_LPC4_tst_pred_w1d_m4hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020


In [13]:
analysis_result_df.to_csv('Alert_Analysis_A_LPC4.csv')
deep_analysis_result_df.to_csv('Alert_Analysis_B_LPC4.csv')

In [ ]:
# Final Evaluation 
# Get the analysis_result_df
# For every model, check if the 'relevant_tags' column contains y and the avg alerts < 15, print selected else print unselected.
